In [1]:
import pandas as pd
import numpy as np
import os

from datetime import datetime
import time
import re


In [2]:
file_path= './Quali_1.xlsx'
df_quali = pd.read_excel(file_path)

In [3]:
# Store file Last Mod Date and Time
modx = os.path.getmtime(file_path)
mod_file_date = datetime.fromtimestamp(modx)

### Filters
1. Only consider funds that currently oprerating
2. Considering Funds that are open for new investments
3. Considering Funds that can be invested by any person, are not exclusive to a party
4. Considering Funds with "Minimum Investment" <> "Not Informed"
5. Removing null benchmark rows

In [4]:
#Applying filters
# 1)
df_quali = df_quali[df_quali['Current Situation'] == "In normal operation"]
df_quali = df_quali[df_quali['Closing Date'].isnull()]
# 2) 
df_quali = df_quali[df_quali['Open to Investments'] == "Yes"]
# 3) 
df_quali = df_quali[df_quali['Qualified Investor'] == "No"]
df_quali = df_quali[df_quali['Exclusive Fund'] == "No"]
# 4)
df_quali = df_quali[df_quali['Minimum Investment'] != "Not informed"]
# 5) 
df_quali = df_quali[~df_quali['Benchmark'].isna()]

### Column Transformation

#### Handling Management Fee (Information resides in 2 columns)
 - Fund Management fee may oscilate, I will consider the .
 - When Management fee (Maximum) is not informed I´ll use regular Management fee.

In [5]:
# logic funtion to use in apply
def transform_fee(row):
    if row['Management Fee (Maximum)'] == "There is not":
        val = row['Management Fee']
    else:
        val = row['Management Fee (Maximum)']
    return val

#Saving results to data Frame
df_quali['Management Fee'] = df_quali.apply(transform_fee, axis=1)


#### Performance Fee 
- Grouping performance fee value if normalized count < 1 %

In [6]:
def group_categories(df,column_name,row_replace_Value = "Other",limit = 0.01):
    mantain_row_vals = []
    for y,x in df[column_name].value_counts(normalize=True).iteritems():
        if x >= limit:
            mantain_row_vals.append(y)
    df[column_name] = df[column_name].map(lambda x: x if x in mantain_row_vals else row_replace_Value)
    return df

In [7]:
# Grouping
df_quali = group_categories(df_quali,'Performance Fee Reference Index',"Other_performance_fee",0.01)

# Changing all Null performance to 0
df_quali['Performance Fee'] = df_quali['Performance Fee'].map(lambda x: 0 if x == "There is not" else x)

#### Benchmark
- Grouping benchmark value if normalized count is < 1%
- Renaming some benchmark typos

In [8]:
# Grouping
df_quali = group_categories(df_quali,'Benchmark',"Other_Benchmark",0.01)

# Typos fix
df_quali['Benchmark'] = df_quali['Benchmark'].str.replace("Not informed","Not Informed")
df_quali['Benchmark'] = df_quali['Benchmark'].str.replace("Not defined","Not Informed")

### Creating column with Fund age in months

In [9]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [10]:
df_quali['age_months'] = df_quali['Start Date of the Fund'].map(lambda x: diff_month(mod_file_date,x))

### Selecting only funds with age > 24 months

In [11]:
df_quali = df_quali[df_quali['age_months']>=24]

### Droping Columns

In [12]:
drop_columns = ['Custodian','Administrator','Director','Headquarter (State)',
                'Headquarters (City)','Headquarters (Neighborhood)',
                'Fee Notes','Management Company','Management Company Name',
                'Management Fee Charging Frequency','Management Fee Provision',
                'Maximum Balance','Minimum Balance','Minimum Movement',
                'Minimum Redemption','Performance Fee Charging Frequency',
                'Performance Fee Criterion','Performance Fee Provision',
                'Performance Fee Water Mark','Purpose of the fund','Quantum Category',
                'Redemption Fee','Redemption Fee Exemption','Redemption NAV',
                'Transactions Notes', 'Type of Disclosure','Fund\'s Length',
                'Closing Date', 'Target Investor','Master',
                'Feeder','Current Situation','Management Fee Type',
                'Open to Investments','Foreign Investment Anbima',
                'Financial Management Fee','Exclusive Fund','Management Fee (Maximum)',
                'Availability of Recovered Resources','Lockup Period','Investment NAV',
                'Join Date','Start Date of the Fund','Qualified Investor'
               ]
df_quali.drop(drop_columns, axis = 1, inplace=True)

In [13]:
df_quali.iloc[9] # Checking single row

Name                               ADAM MACRO 1 FIC MULTIMERCADO
Benchmark                                   CDI (Interbank Rate)
CNPJ                                          24.119.419/0001-25
CVM Category                                         Multimarket
Entidade de Previdência 3922_US                               No
Entidade de Previdência 4444_US                               No
Entidade de Previdência 4661_US                               No
Legal Classification                                         FIC
Leveraged Anbima                                             Yes
Management Fee                                             0.025
Minimum Investment                                         10000
Performance Fee                                              0.2
Performance Fee Reference Index                      100% do CDI
Portfolio Manager                                   Márcio Appel
Private Credit Anbima                                         No
Tax Classification       

### Creating Dummies engineering

In [14]:
columns_to_dummy = ['Tax Classification','Private Credit Anbima','Leveraged Anbima',
                   'Legal Classification','Entidade de Previdência 3922_US',                                                  
                    'Entidade de Previdência 4444_US','Entidade de Previdência 4661_US',
                    'CVM Category','Performance Fee Reference Index','Benchmark']
df_quali = pd.get_dummies(df_quali, columns=columns_to_dummy,drop_first=True)

# EDA Quantitative Features

In [15]:
file_path= './Quant_4.xlsx'
df_quant = pd.read_excel(file_path)

### Renaming Columns

In [16]:
# Function to correct col names
def fix_col_names(col_str):
    if col_str == "CNPJ":
        return col_str     
    elif col_str[::-1][1]==".":
        replaced = col_str.replace('.1', '_12m')
        replaced = replaced.replace('.2', '_24m')
    else:
        replaced = col_str + '_6m'     
#     print(col_str[::-1][1],col_str,replaced)
    return replaced

In [17]:
# Renaming 2 last assets col
df_quant.rename({'Patrimônio Líquido final da série': 'Last_Assets',
                 'Patrimônio Líquido final da série.1': 'Last_Assets_date'},
                axis='columns',inplace=True)
# Adding time period to column names
cols = df_quant.columns
df_quant.columns = [fix_col_names(cols[pos]).replace(" ","_") 
                    if pos > 2 else cols[pos].replace(" ","_")  
                    for pos in range(len(cols))]


In [18]:
df_quant.columns # Checking column names

Index(['Name', 'Last_Assets', 'Last_Assets_date', 'Assets_Flow_6m',
       'Assets_Flow_12m', 'Assets_Flow_24m', 'Average_Assets_6m',
       'Average_Assets_12m', 'Average_Assets_24m', 'Return_6m', 'Return_12m',
       'Return_24m', 'Volatility_6m', 'Volatility_12m', 'Volatility_24m',
       'Excess_Return_-_CDI_Opening_6m', 'Excess_Return_-_CDI_Opening_12m',
       'Excess_Return_-_CDI_Opening_24m', 'Excess_Return_-_Ibovespa_6m',
       'Excess_Return_-_Ibovespa_12m', 'Excess_Return_-_Ibovespa_24m',
       'Excess_Return_-_Dollar_6m', 'Excess_Return_-_Dollar_12m',
       'Excess_Return_-_Dollar_24m', 'CNPJ'],
      dtype='object')

### Filtering & Merging & Fixing Data Types
1. Only funds selected in the Qualitative Data Frame
2. Rows with no null value.
3. Creating Column with Manager Name
4. Converting to numeric

In [19]:
#1)
df_quant = df_quant[df_quant['CNPJ'].isin(df_quali['CNPJ'])]
# df_quant.isnull().sum().sum() # null check

#2)
df_quant = df_quant[~df_quant['Excess_Return_-_CDI_Opening_24m'].isnull()]
# df_quant.isnull().sum().sum()   # null check

# 3)
df_quant = pd.merge(df_quant, df_quali[['CNPJ','Portfolio Manager']], on='CNPJ')

# 4)
string_columns = ["Last_Assets_date","Name","CNPJ","Portfolio Manager"]
numeric_cols = df_quant[df_quant.columns[~df_quant.columns.isin(string_columns)]].columns
df_quant[numeric_cols] = df_quant[numeric_cols].apply(pd.to_numeric)
# df_quant.dtypes  # dtypes check

### Feature Engineering columns with Manager Information

In [20]:
managers = pd.DataFrame(df_quant['Portfolio Manager'].value_counts())
managers.columns = ["#_Funds_managed"]

In [21]:
x = df_quant[df_quant['Portfolio Manager']=='Eduardo Alves de Castro']['Last_Assets'].values

In [22]:
managers['Manager_avg_Assets'] = [np.average(df_quant[df_quant['Portfolio Manager']
                         ==manage_name]['Last_Assets'].values) for manage_name
                         in managers.index.values]

In [24]:
df_quant["#_Funds_managed"] = [managers.loc[manager]['#_Funds_managed'] for manager
                              in df_quant['Portfolio Manager']]

In [26]:
df_quant["Manager_avg_Assets"] = [managers.loc[manager]['Manager_avg_Assets'] for manager
                              in df_quant['Portfolio Manager']]

# Joining Qualitative and Quantitative Data Frames

In [27]:
df_quant.shape # Dimention check

(1380, 28)

In [28]:
df_quali.shape # Dimention check

(1395, 34)

In [29]:
df_quant.drop(['Portfolio Manager','Name'], axis = 1, inplace=True)
final_df = pd.merge(df_quali,df_quant,on = 'CNPJ')

In [30]:
final_df.shape # Dimention check

(1380, 59)

## Saving final csv

In [32]:
final_df.to_csv('./clean_df.csv',index=False)